In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           Var,
                           Reals,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from HFCs_bmimBF4_PR import configuration
from R125_bmimBF4_PR import configuration as configuration_R125
from R32_bmimBF4_PR import configuration as configuration_R32

# Load data from csv
data = pd.read_csv('HFCSBF4_g_all.csv')

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})
    
    m.fs.a = Var(within=Reals, bounds=(-6,6), initialize=-0.3)
    m.fs.b = Var(within=Reals, bounds=(-6,6), initialize=-0.02)
    m.fs.c = Var(within=Reals, bounds=(-6,6), initialize=0.1)
    m.fs.d = Var(within=Reals, bounds=(-6,6), initialize=0.06)

    eps1 = 0.05
    eps2 = 0.1
    if  float(data["x_R32"]) < 0.0001:

        m.fs.properties = GenericParameterBlock(default=configuration_R125)

        m.fs.state_block = m.fs.properties.state_block_class(
            default={"parameters": m.fs.properties,
                     "defined_state": True})
        x = float(data["x_R125"])+eps1
        m.fs.state_block.flow_mol.fix(1)
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.pressure.fix(float(data["pressure"]))
        m.fs.state_block.mole_frac_comp["bmimBF4"].fix(1-x)
        m.fs.state_block.mole_frac_comp["R125"].fix(x)

        m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].fix(1.6)
        m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].fix(0.06)

        m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

        m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].unfix()
        m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].unfix()
        
        # Fix the state variables on the state block
        m.fs.state_block.pressure.unfix()
        m.fs.state_block.mole_frac_comp["bmimBF4"].unfix()
        m.fs.state_block.mole_frac_comp["R125"].unfix()
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', "R125"].fix(float(data["x_R125"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', "bmimBF4"].fix(float(data["x_IL"]))
#         m.fs.state_block.mole_frac_phase_comp['Liq', "bmimBF4"].unfix()
        m.fs.state_block.mole_frac_comp["R125"].fix(float(data["x_R125"])+eps1)
        m.fs.state_block.mole_frac_comp["bmimBF4"].unfix()

        # Set bounds on variables to be estimated
        m.fs.properties.PR_kappa_A["bmimBF4", "R125"].setlb(-5)
        m.fs.properties.PR_kappa_A["bmimBF4", "R125"].setub(5)

        m.fs.properties.PR_kappa_A["R125", "bmimBF4"].setlb(-5)
        m.fs.properties.PR_kappa_A["R125", "bmimBF4"].setub(5)
        
        m.fs.A = Constraint(expr= m.fs.a == m.fs.properties.PR_kappa_A["bmimBF4", "R125"])
        m.fs.B = Constraint(expr= m.fs.b == m.fs.properties.PR_kappa_A["R125", "bmimBF4"])

    elif  float(data["x_R125"]) < 0.0001:

        m.fs.properties = GenericParameterBlock(default=configuration_R32)

        m.fs.state_block = m.fs.properties.state_block_class(
            default={"parameters": m.fs.properties,
                     "defined_state": True})
        x = float(data["x_R32"])+eps2
        m.fs.state_block.flow_mol.fix(1)
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.pressure.fix(float(data["pressure"]))
        m.fs.state_block.mole_frac_comp["bmimBF4"].fix(1-x)
        m.fs.state_block.mole_frac_comp["R32"].fix(x)

        # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
        m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].fix(-0.046599)
        m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].fix(-0.37995)

        m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

        m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].unfix()
        m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].unfix()
        
        # Fix the state variables on the state block
        m.fs.state_block.pressure.unfix()
        m.fs.state_block.mole_frac_comp["bmimBF4"].unfix()
        m.fs.state_block.mole_frac_comp["R32"].unfix()
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', "R32"].fix(float(data["x_R32"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', "bmimBF4"].fix(float(data["x_IL"]))
#         m.fs.state_block.mole_frac_phase_comp['Liq', "bmimBF4"].unfix()
        m.fs.state_block.mole_frac_comp["R32"].fix(float(data["x_R32"])+eps2)
        m.fs.state_block.mole_frac_comp["bmimBF4"].unfix()

        # Set bounds on variables to be estimated
        m.fs.properties.PR_kappa_A["bmimBF4", "R32"].setlb(-5)
        m.fs.properties.PR_kappa_A["bmimBF4", "R32"].setub(5)

        m.fs.properties.PR_kappa_A["R32", "bmimBF4"].setlb(-5)
        m.fs.properties.PR_kappa_A["R32", "bmimBF4"].setub(5)
        
        m.fs.C = Constraint(expr= m.fs.c == m.fs.properties.PR_kappa_A["bmimBF4", "R32"])
        m.fs.D = Constraint(expr= m.fs.d == m.fs.properties.PR_kappa_A["R32", "bmimBF4"])

    else:

        m.fs.properties = GenericParameterBlock(default=configuration)

        m.fs.state_block = m.fs.properties.state_block_class(
            default={"parameters": m.fs.properties,
                     "defined_state": True})

        m.fs.state_block.flow_mol.fix(1)
        m.fs.state_block.temperature.fix(298.15)
        m.fs.state_block.pressure.fix(100000)
        m.fs.state_block.mole_frac_comp["R32"].fix(0.02)
        m.fs.state_block.mole_frac_comp["R125"].fix(0.02)
        m.fs.state_block.mole_frac_comp["bmimBF4"].fix(0.96)

        m.fs.properties.PR_kappa_A['R32', 'R125'].fix(0.009)
        m.fs.properties.PR_kappa_A['R125', 'R32'].fix(0.007)
        m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].fix(-0.02)
        m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].fix(-0.37995)
        m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].fix(1.6)
        m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].fix(0.06)

   
        # Initialize the flash unit
        m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

        # Fix at actual temperature

        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.pressure.unfix()
        m.fs.state_block.mole_frac_comp["R125"].unfix()
        m.fs.state_block.mole_frac_comp["R32"].unfix()
        m.fs.state_block.mole_frac_comp["bmimBF4"].unfix()
#         m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#         m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))   
        m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"])) 
        m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimBF4'].fix(float(data["x_IL"])) 

        m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].unfix()
        m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].unfix()
        m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].unfix()
        m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].unfix()
        
        # Set bounds on variables to be estimated
        m.fs.properties.PR_kappa_A['R32', 'R125'].setlb(-10)
        m.fs.properties.PR_kappa_A['R32', 'R125'].setub(10)

        m.fs.properties.PR_kappa_A['R125', 'R32'].setlb(-10)
        m.fs.properties.PR_kappa_A['R125', 'R32'].setub(10)

        m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].setlb(-10)
        m.fs.properties.PR_kappa_A['R32', 'bmimBF4'].setub(10)

        m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].setlb(-10)
        m.fs.properties.PR_kappa_A['bmimBF4', 'R32'].setub(10)

        m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].setlb(-10)
        m.fs.properties.PR_kappa_A['bmimBF4', 'R125'].setub(10)

        m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].setlb(-10)
        m.fs.properties.PR_kappa_A['R125', 'bmimBF4'].setub(10)
        
        m.fs.A = Constraint(expr= m.fs.a == m.fs.properties.PR_kappa_A["bmimBF4", "R125"])
        m.fs.B = Constraint(expr= m.fs.b == m.fs.properties.PR_kappa_A["R125", "bmimBF4"])
        m.fs.C = Constraint(expr= m.fs.c == m.fs.properties.PR_kappa_A["bmimBF4", "R32"])
        m.fs.D = Constraint(expr= m.fs.d == m.fs.properties.PR_kappa_A["R32", "bmimBF4"])
        
        # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 100000, "x_R32":0.000001, "x_IL":0.7312,  
             "x_R125":0.5, "y_R125":0.2501, "y_R32":0.7499, "y_IL":0.7312}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

2022-03-07 16:31:31 [INFO] idaes.init.fs.state_block: Property package initialization: infeasible - Converged to a locally infeasible point. Problem may be infeasible..
The initial DOF is 2


In [7]:
variable_name = [
                "fs.a",
                "fs.b",
                "fs.c",
                "fs.d"]

In [8]:
# def SSE(m, data):
#     expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
#     return expr*1e-9

def SSE(m, data):
    if  float(data["x_R32"]) < 0.0001:
        expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
        return expr*1e-9*2
    elif  float(data["x_R125"]) < 0.0001:
        expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
        return expr*1e-9
    else:
        expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
        return expr*1e-9*10*1.9

In [9]:
solver_options = {"max_iter": 10}
pest = parmest.Estimator(PR_model, data, variable_name, SSE, solver_options)
# solver_options = {'tol': 1e-6, 'max_iter':100}

obj_value, parameters = pest.theta_est()

2022-03-07 16:31:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:31:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:31:48 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:31:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:31:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:31:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:31:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:31:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-07 16:31:57 [INFO] idaes

  44  4.1843331e+01 4.29e+00 2.68e+07  -1.0 4.19e+07    -  5.88e-02 1.90e-03H  1
  45  3.6427826e+01 4.28e+00 6.92e+07  -1.0 3.72e+07    -  3.85e-02 2.44e-03H  1
  46  3.8530666e+01 4.27e+00 1.35e+08  -1.0 4.00e+07    -  6.16e-05 2.90e-03h  1
  47  3.6328112e+01 4.27e+00 1.67e+08  -1.0 4.59e+07    -  3.58e-02 1.19e-03H  1
  48  3.7821736e+01 4.26e+00 2.30e+08  -1.0 4.14e+07    -  2.44e-05 1.69e-03h  6
  49  3.6326021e+01 4.26e+00 2.82e+08  -1.0 4.29e+07    -  3.89e-02 8.76e-04H  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  50  3.9670644e+01 4.24e+00 1.05e+09  -1.0 4.09e+07    -  1.75e-05 3.31e-03h  5
  51  4.0929951e+01 4.24e+00 1.10e+09  -1.0 3.97e+07    -  3.81e-03 1.01e-03h  7
  52  4.1279282e+01 4.24e+00 1.11e+09  -1.0 3.96e+07    -  5.07e-03 2.67e-04h  9
  53  4.1368807e+01 4.24e+00 1.11e+09  -1.0 4.02e+07    -  1.71e-02 6.68e-05h 11
  54  4.1391042e+01 4.24e+00 1.11e+09  -1.0 4.42e+07    -  1.18e-02 1.54e-05h 13
  55r 4.1391042e+01 4.24e+00

In [10]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 13.843153

The values for the parameters are as follows:
fs.a = 0.7908048724984598
fs.b = 0.059448669291419835
fs.c = 0.5425182159563849
fs.d = 0.031326150060040636


In [11]:
print(large_residuals_set(pest.ef_instance))

ComponentSet([])


In [12]:
from idaes.core.util.model_statistics import report_statistics
print(report_statistics(pest.ef_instance))


Model Statistics  -  _Q_opt 

Degrees of Freedom: 7 

Total No. Variables: 2307 
    No. Fixed Variables: 1551
    No. Unused Variables: 642 (Fixed):642)
    No. Variables only in Inequalities: 0 (Fixed: 0) 

Total No. Constraints: 749 
    No. Equality Constraints: 749 (Deactivated: 0)
    No. Inequality Constraints: 0 (Deactivated: 0)

No. Objectives: 42 (Deactivated: 41)

No. Blocks: 334 (Deactivated: 0) 
No. Expressions: 1465 

None
